In [1]:
import pandas as pd
df_train=pd.read_csv('dataset/yds_train2018.csv')
df_test=pd.read_csv('dataset/yds_test2018.csv')
df_train.head()

S_No  Year  Month  Week Merchant_ID  Product_ID    Country     Sales
0     1  2013      1     1     ar00001           1  Argentina  157500.0
1     2  2013      1     1     ar00003           1  Argentina   39375.0
2     3  2013      1     1     ar00004           1  Argentina   15750.0
3     4  2013      1     1     ar00007           1  Argentina   47250.0
4     5  2013      1     1     ar00008           1  Argentina  283500.0

In [2]:
df_train['Merchant_ID'].value_counts().count()

1829

In [3]:
Dates=[]
length=len(df_train['Year'])
for i in range(79072):
    Dates.append(str(df_train['Year'][i])+'-'+str(df_train['Month'][i]))
    

df_train['Dates']=Dates
df_train['Dates']=pd.to_datetime(df_train['Dates'])
df_train['Dates'].head()
df_train.head(2)

S_No  Year  Month  Week Merchant_ID  Product_ID    Country     Sales  \
0     1  2013      1     1     ar00001           1  Argentina  157500.0   
1     2  2013      1     1     ar00003           1  Argentina   39375.0   

       Dates  
0 2013-01-01  
1 2013-01-01

In [4]:
data_monthly=df_train.groupby(['Year', 'Month', 'Product_ID', 'Country'],as_index=False)['Sales'].sum()
data_monthly.head(2)

Year  Month  Product_ID    Country        Sales
0  2013      1           1  Argentina  34346025.00
1  2013      1           1   Columbia   3947356.31

In [5]:
import numpy as np
Unique_Countries=['Finland']
Unique_Countries

['Finland']

In [6]:
ydsnaiveSumission=pd.DataFrame()
trainForecastlastpoint=[]
df_final=pd.DataFrame()

In [7]:
for country in Unique_Countries:
    data_month_country=data_monthly[data_monthly['Country']==country]
    UniqProduct=[4]
    #UniqProduct=list(np.unique(data_monthly['Product_ID']))
    for product in UniqProduct:
        data_month_filtered=data_month_country[data_month_country['Product_ID']==product]
        lastPointValue=data_month_filtered['Sales'].iloc[-1]
        submission_filter=df_test[df_test['Country']==country]
        submission_filter=submission_filter[submission_filter['Product_ID']==product]
        horizon=submission_filter.shape[0]
        submission_filter['Sales']=lastPointValue
        ydsnaiveSumission=ydsnaiveSumission.append(submission_filter,ignore_index=True)
        ydsnaiveSumission=ydsnaiveSumission.drop_duplicates(keep='first',inplace=False)
        #dmfs=data_month_filtered['Sales']
        #lpv=lastPointValue
        #temp={'Sales':dmfs,'lastPointValue':lpv}
        #df_temp=pd.DataFrame(temp)
        #trainForecastlastpoint.append(df_temp)

ydsnaiveSumission

S_No  Year  Month  Product_ID  Country        Sales
0  79169  2015     11           4  Finland  31268810.86
1  79170  2015     12           4  Finland  31268810.86
2  79171  2016      1           4  Finland  31268810.86
3  79172  2016      2           4  Finland  31268810.86
4  79173  2016      3           4  Finland  31268810.86
5  79174  2016      4           4  Finland  31268810.86
6  79175  2016      5           4  Finland  31268810.86
7  79176  2016      6           4  Finland  31268810.86
8  79177  2016      7           4  Finland  31268810.86

In [8]:
df_a=pd.read_csv('Argentina.csv')
df_b=pd.read_csv('Belgium.csv')
df_c=pd.read_csv('Columbia.csv')
df_d=pd.read_csv('Denmark.csv')
df_e=pd.read_csv('England.csv')
df_f=pd.read_csv('Finland.csv')

In [9]:
df_h=pd.concat([df_a,df_b],ignore_index=True)
df_h=pd.concat([df_h,df_c],ignore_index=True)
df_h=pd.concat([df_h,df_d],ignore_index=True)
df_h=pd.concat([df_h,df_e],ignore_index=True)
df_h=pd.concat([df_h,df_f],ignore_index=True)
df_h.head()

Unnamed: 0   S_No  Year  Month  Product_ID    Country       Sales
0           0  79073  2016      4           1  Argentina  15016050.0
1           1  79074  2016      5           1  Argentina  15016050.0
2           2  79075  2016      6           1  Argentina  15016050.0
3           3  79076  2016      7           1  Argentina  15016050.0
4           4  79077  2016      8           1  Argentina  15016050.0

In [10]:
df_h=df_h.drop(['Unnamed: 0'],axis=1)

In [11]:
df_h=pd.get_dummies(df_h,columns=['Country'],drop_first=True)

In [12]:
df_test=df_test.drop(['Sales'],axis=1)

In [13]:
df_new_test=df_test
df_new_test=pd.get_dummies(df_new_test,columns=['Country'],drop_first=True)


In [14]:
X=df_h.values
y=df_h['Sales']
test=df_new_test.values

In [15]:
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor(random_state = 42)
from pprint import pprint
print('Parameters currently in use:\n')
pprint(rf.get_params())

Parameters currently in use:

{'bootstrap': True,
 'criterion': 'mse',
 'max_depth': None,
 'max_features': 'auto',
 'max_leaf_nodes': None,
 'min_impurity_decrease': 0.0,
 'min_impurity_split': None,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 10,
 'n_jobs': 1,
 'oob_score': False,
 'random_state': 42,
 'verbose': 0,
 'warm_start': False}


In [16]:
from sklearn.model_selection import RandomizedSearchCV
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
max_features = ['auto', 'sqrt']
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
min_samples_split = [2, 5, 10]
min_samples_leaf = [1, 2, 4]
bootstrap = [True, False]
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
pprint(random_grid)

{'bootstrap': [True, False],
 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, None],
 'max_features': ['auto', 'sqrt'],
 'min_samples_leaf': [1, 2, 4],
 'min_samples_split': [2, 5, 10],
 'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000]}


In [17]:

rf = RandomForestRegressor()
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = -1)
rf_random.fit(X,y)

Fitting 3 folds for each of 100 candidates, totalling 300 fits
[CV] n_estimators=400, min_samples_split=5, min_samples_leaf=1, max_features=sqrt, max_depth=30, bootstrap=True 
[CV] n_estimators=400, min_samples_split=5, min_samples_leaf=1, max_features=sqrt, max_depth=30, bootstrap=True 
[CV] n_estimators=400, min_samples_split=5, min_samples_leaf=1, max_features=sqrt, max_depth=30, bootstrap=True 
[CV] n_estimators=2000, min_samples_split=5, min_samples_leaf=1, max_features=sqrt, max_depth=10, bootstrap=True 
[CV]  n_estimators=400, min_samples_split=5, min_samples_leaf=1, max_features=sqrt, max_depth=30, bootstrap=True, total=   1.0s
[CV] n_estimators=2000, min_samples_split=5, min_samples_leaf=1, max_features=sqrt, max_depth=10, bootstrap=True 
[CV]  n_estimators=400, min_samples_split=5, min_samples_leaf=1, max_features=sqrt, max_depth=30, bootstrap=True, total=   1.0s
[CV] n_estimators=2000, min_samples_split=5, min_samples_leaf=1, max_features=sqrt, max_depth=10, bootstrap=True 


[CV] n_estimators=800, min_samples_split=5, min_samples_leaf=1, max_features=sqrt, max_depth=90, bootstrap=False 
[CV]  n_estimators=1800, min_samples_split=5, min_samples_leaf=4, max_features=auto, max_depth=10, bootstrap=False, total=   5.3s
[CV] n_estimators=2000, min_samples_split=10, min_samples_leaf=1, max_features=sqrt, max_depth=10, bootstrap=False 


[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:   27.1s


[CV]  n_estimators=800, min_samples_split=5, min_samples_leaf=1, max_features=sqrt, max_depth=90, bootstrap=False, total=   2.5s
[CV] n_estimators=2000, min_samples_split=10, min_samples_leaf=1, max_features=sqrt, max_depth=10, bootstrap=False 
[CV]  n_estimators=800, min_samples_split=5, min_samples_leaf=1, max_features=sqrt, max_depth=90, bootstrap=False, total=   2.0s
[CV] n_estimators=2000, min_samples_split=10, min_samples_leaf=1, max_features=sqrt, max_depth=10, bootstrap=False 
[CV]  n_estimators=800, min_samples_split=5, min_samples_leaf=1, max_features=sqrt, max_depth=90, bootstrap=False, total=   2.0s
[CV] n_estimators=1600, min_samples_split=5, min_samples_leaf=2, max_features=sqrt, max_depth=10, bootstrap=False 
[CV]  n_estimators=2000, min_samples_split=10, min_samples_leaf=1, max_features=sqrt, max_depth=10, bootstrap=False, total=   5.1s
[CV] n_estimators=1600, min_samples_split=5, min_samples_leaf=2, max_features=sqrt, max_depth=10, bootstrap=False 
[CV]  n_estimators=1

[CV] n_estimators=1800, min_samples_split=2, min_samples_leaf=2, max_features=auto, max_depth=None, bootstrap=True 
[CV]  n_estimators=1400, min_samples_split=5, min_samples_leaf=4, max_features=sqrt, max_depth=80, bootstrap=True, total=   3.6s
[CV] n_estimators=1800, min_samples_split=2, min_samples_leaf=2, max_features=auto, max_depth=None, bootstrap=True 
[CV]  n_estimators=1400, min_samples_split=5, min_samples_leaf=4, max_features=sqrt, max_depth=80, bootstrap=True, total=   5.1s
[CV] n_estimators=1400, min_samples_split=5, min_samples_leaf=1, max_features=sqrt, max_depth=80, bootstrap=False 
[CV]  n_estimators=1800, min_samples_split=2, min_samples_leaf=2, max_features=auto, max_depth=None, bootstrap=True, total=   6.3s
[CV] n_estimators=1400, min_samples_split=5, min_samples_leaf=1, max_features=sqrt, max_depth=80, bootstrap=False 
[CV]  n_estimators=1800, min_samples_split=2, min_samples_leaf=2, max_features=auto, max_depth=None, bootstrap=True, total=   6.5s
[CV] n_estimators=

...

[CV]  n_estimators=1800, min_samples_split=5, min_samples_leaf=2, max_features=sqrt, max_depth=60, bootstrap=True, total=   4.7s
[CV] n_estimators=400, min_samples_split=10, min_samples_leaf=2, max_features=sqrt, max_depth=90, bootstrap=False 
[CV]  n_estimators=400, min_samples_split=10, min_samples_leaf=2, max_features=sqrt, max_depth=90, bootstrap=False, total=   0.9s
[CV] n_estimators=400, min_samples_split=10, min_samples_leaf=2, max_features=sqrt, max_depth=90, bootstrap=False 
[CV]  n_estimators=400, min_samples_split=10, min_samples_leaf=2, max_features=sqrt, max_depth=90, bootstrap=False, total=   1.0s
[CV] n_estimators=400, min_samples_split=10, min_samples_leaf=2, max_features=sqrt, max_depth=90, bootstrap=False 
[CV]  n_estimators=1600, min_samples_split=5, min_samples_leaf=1, max_features=auto, max_depth=90, bootstrap=True, total=   5.0s
[CV] n_estimators=1600, min_samples_split=10, min_samples_leaf=1, max_features=sqrt, max_depth=None, bootstrap=True 
[CV]  n_estimators=4

[CV] n_estimators=400, min_samples_split=5, min_samples_leaf=1, max_features=sqrt, max_depth=100, bootstrap=True 
[CV]  n_estimators=600, min_samples_split=10, min_samples_leaf=1, max_features=sqrt, max_depth=40, bootstrap=False, total=   1.5s
[CV] n_estimators=400, min_samples_split=5, min_samples_leaf=1, max_features=sqrt, max_depth=100, bootstrap=True 
[CV]  n_estimators=800, min_samples_split=5, min_samples_leaf=4, max_features=sqrt, max_depth=70, bootstrap=False, total=   2.0s
[CV] n_estimators=400, min_samples_split=5, min_samples_leaf=1, max_features=sqrt, max_depth=100, bootstrap=True 
[CV]  n_estimators=600, min_samples_split=10, min_samples_leaf=1, max_features=sqrt, max_depth=40, bootstrap=False, total=   1.8s
[CV] n_estimators=1600, min_samples_split=2, min_samples_leaf=4, max_features=sqrt, max_depth=80, bootstrap=True 
[CV]  n_estimators=400, min_samples_split=5, min_samples_leaf=1, max_features=sqrt, max_depth=100, bootstrap=True, total=   1.2s
[CV] n_estimators=1600, mi

[CV] n_estimators=200, min_samples_split=5, min_samples_leaf=4, max_features=auto, max_depth=100, bootstrap=False 
[CV]  n_estimators=200, min_samples_split=5, min_samples_leaf=4, max_features=auto, max_depth=100, bootstrap=False, total=   0.5s
[CV] n_estimators=200, min_samples_split=5, min_samples_leaf=4, max_features=auto, max_depth=100, bootstrap=False 
[CV]  n_estimators=200, min_samples_split=5, min_samples_leaf=4, max_features=auto, max_depth=100, bootstrap=False, total=   0.5s
[CV] n_estimators=2000, min_samples_split=2, min_samples_leaf=2, max_features=sqrt, max_depth=20, bootstrap=True 
[CV]  n_estimators=1000, min_samples_split=2, min_samples_leaf=1, max_features=auto, max_depth=10, bootstrap=False, total=   2.3s
[CV] n_estimators=2000, min_samples_split=2, min_samples_leaf=2, max_features=sqrt, max_depth=20, bootstrap=True 
[CV]  n_estimators=1000, min_samples_split=2, min_samples_leaf=1, max_features=auto, max_depth=10, bootstrap=False, total=   2.4s
[CV] n_estimators=2000

[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed:  4.2min finished


RandomizedSearchCV(cv=3, error_score='raise',
          estimator=RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
           oob_score=False, random_state=None, verbose=0, warm_start=False),
          fit_params=None, iid=True, n_iter=100, n_jobs=-1,
          param_distributions={'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000], 'max_features': ['auto', 'sqrt'], 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, None], 'min_samples_split': [2, 5, 10], 'min_samples_leaf': [1, 2, 4], 'bootstrap': [True, False]},
          pre_dispatch='2*n_jobs', random_state=42, refit=True,
          return_train_score='warn', scoring=None, verbose=2)

In [18]:
rf_random.best_params_

{'bootstrap': True,
 'max_depth': 100,
 'max_features': 'auto',
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'n_estimators': 800}

In [21]:
from sklearn.ensemble import RandomForestRegressor
regressor = RandomForestRegressor(bootstrap=True,max_depth=None,max_features='auto',min_samples_leaf=2,min_samples_split=5,n_estimators=800)
regressor.fit(X,y)
regressor_pred=regressor.predict(test)
df_test['Sales']=regressor_pred
df_test[['S_No','Year','Month','Product_ID','Country','Sales']].to_csv('final_submission_5.csv', index=False)

In [22]:
df_result=pd.read_csv('final_submission_5.csv')
df_result

S_No  Year  Month  Product_ID    Country         Sales
0    79073  2016      4           1  Argentina  5.494023e+06
1    79074  2016      5           1  Argentina  5.486178e+06
2    79075  2016      6           1  Argentina  5.486178e+06
3    79076  2016      7           1  Argentina  5.483738e+06
4    79077  2016      8           1  Argentina  5.483738e+06
5    79078  2016      9           1  Argentina  5.483738e+06
6    79079  2016     10           1  Argentina  5.483738e+06
7    79080  2016     11           1  Argentina  5.483738e+06
8    79081  2016     12           1  Argentina  5.483738e+06
9    79082  2017      1           1  Argentina  5.557687e+06
10   79083  2017      2           1  Argentina  5.542563e+06
11   79084  2017      3           1  Argentina  5.490327e+06
12   79085  2016      4           2  Argentina  5.127318e+06
13   79086  2016      5           2  Argentina  5.093035e+06
14   79087  2016      6           2  Argentina  5.093035e+06
15   79088  2016      7           2  Argentina  5.090596e+06
16   79089  2016      8           2  Argentina  5.090596e+06
17   79090  2016      9           2  Argentina  5.090596e+06
18   79091  2016     10           2  Argentina  5.090596e+06
19   79092  2016     11           2  Argentina  5.090596e+06
20   79093  2016     12           2  Argentina  5.090596e+06
21   79094  2017      1           2  Argentina  5.166374e+06
22   79095  2017      2           2  Argentina  5.146509e+06
23   79096  2017      3           2  Argentina  5.109513e+06
24   79097  2017      1           3  Argentina  5.166506e+06
25   79098  2017      2           3  Argentina  4.579457e+06
26   79099  2017      3           3  Argentina  2.776070e+06
27   79100  2016      4           2    Belgium  2.232865e+06
28   79101  2016      5           2    Belgium  2.190978e+06
29   79102  2016      6           2    Belgium  2.190978e+06
..     ...   ...    ...         ...        ...           ...
75   79148  2017      1           2    Denmark  3.567106e+06
76   79149  2017      2           2    Denmark  3.537345e+06
77   79150  2017      3           2    Denmark  3.482336e+06
78   79151  2015     11           5    England  9.885657e+07
79   79152  2015     12           5    England  9.885657e+07
80   79153  2016      1           5    England  9.916101e+07
81   79154  2016      2           5    England  1.019887e+08
82   79155  2016      3           5    England  1.021711e+08
83   79156  2016      4           5    England  1.021641e+08
84   79157  2016      5           5    England  1.021589e+08
85   79158  2016      6           5    England  1.021589e+08
86   79159  2016      7           5    England  1.021589e+08
87   79160  2015     11           4    England  1.021589e+08
88   79161  2015     12           4    England  1.021589e+08
89   79162  2016      1           4    England  1.022004e+08
90   79163  2016      2           4    England  1.021924e+08
91   79164  2016      3           4    England  1.021711e+08
92   79165  2016      4           4    England  1.021641e+08
93   79166  2016      5           4    England  1.021589e+08
94   79167  2016      6           4    England  1.021589e+08
95   79168  2016      7           4    England  1.021589e+08
96   79169  2015     11           4    Finland  2.744249e+07
97   79170  2015     12           4    Finland  2.744249e+07
98   79171  2016      1           4    Finland  2.747219e+07
99   79172  2016      2           4    Finland  2.746670e+07
100  79173  2016      3           4    Finland  2.745243e+07
101  79174  2016      4           4    Finland  2.744523e+07
102  79175  2016      5           4    Finland  2.744249e+07
103  79176  2016      6           4    Finland  2.744249e+07
104  79177  2016      7           4    Finland  2.744249e+07

[105 rows x 6 columns]